In [1]:
import matplotlib.pyplot as plt
import numpy as np
from utils import *
import time

# Load the datapoints into X. The i-th line of matrix X corresponds to the embedding vector of node i.

In [2]:
# Specify the file path
file_path = 'datasets/all_data.txt'  # Replace with the actual file path

# Initialize an empty list to store the matrices
matrices = []

# Read the file line by line
with open(file_path, 'r') as file:
    for line in file:
        # Extract numerical values and create the embedding
        embedding = [float(element.split(":")[1]) for element in " ".join(line.split(" ")[1:]).split()]
        
        # Append the embedding to the matrices list
        matrices.append(embedding)
X = np.array(matrices)

In [4]:
sx = np.sum(X**2, axis=1, keepdims=True)
sy = np.sum(X**2, axis=1, keepdims=True)
distances = -2 * X.dot(X.T) + sx + sy.T
distances[distances < 1] = 0
distances = np.sqrt(distances)

### We analyze the impact of the density of the graph to running time

In [9]:
anchor_probability_numerator = 20.
notifications = 2
epsilon = 0.2
deletion_prob_at_each_step = 0.2
num_samples_for_agreement_and_heavyness_calculation = 2
num_samples_for_connect_procedure = 2

In [12]:
thresholds = [np.mean(distances)/10.0, np.mean(distances)/15.0, np.mean(distances)/20.0, np.mean(distances)/25.0, np.mean(distances)/30.0]


n, m = distances.shape

num_experiments = 1

for threshold in thresholds:
    adjacency_lists = {}
    # self loops are added because diagonal elements have a value of 1
    for a in range(0,n):
        for b in range(0,m):
            if distances[a,b] > threshold:
                continue
            # Add edge to the adjacency lists
            if a not in adjacency_lists:
                adjacency_lists[a] = OptList()
            adjacency_lists[a].append(b)

            if b not in adjacency_lists:
                adjacency_lists[b] = OptList()
            adjacency_lists[b].append(a)
            
    # calculate the objective of the classical pivot on the entire graph
    classical_pivot_clustering_sol = classical_pivot(graph_adjacency_lists)
    classical_pivot_clustering = correlation_clustering_value(current_graph, classical_pivot_clustering_sol)

    
    # now we have created a graph

    # These are the statistics of nodes and edges
    nodes = len(adjacency_lists.keys())
    edges = sum([len(adjacency_lists[node]) for node in adjacency_lists.keys()])
    density = edges/nodes

    print(f"number of nodes = {nodes}, number of edges {edges}==> density = {density}")

    graph_adjacency_lists = adjacency_lists
    
    # this it to measure the time in each execution
    dynamic_pivot_total_time = .0
    agreement_total_time = .0
    dynamic_pivot_worst_update_time = .0
    agreement_worst_update_time = .0

    agreement_total_time_node_additions = .0
    agreement_total_time_node_deletions = .0

    agreement_worst_update_time_node_additions = .0
    agreement_worst_update_time_node_deletions = .0
    
     # Create lists to store objective values for each dataset
    correlation_values_clustering = []
    correlation_values_all_singletons = []
    correlation_values_pivot_clustering = []


    for t in range(0, num_experiments):
        random.seed(t)

        # Generate a random node arrival order
        random_node_order = generate_random_node_order(graph_adjacency_lists)

        # Initialization for the random variables of dynamic pivot
        # we first contract a random permulation pi: node --> order in the random permulation
        pi = {element: index for index, element in enumerate(generate_random_node_order(graph_adjacency_lists))}
        eta = {key: key for key in pi.keys()}
        pivot_clustering = {key: key for key in pi.keys()}

        #Initializaiton for the random variables of agreement
        sparse_graph = {}
        nodes_present_at_the_moment = OptList()
        Phi = set()
        Phi_nodes = {}
        I_nodes = defaultdict(set)
        B_nodes = {node:OptList() for node in graph_adjacency_lists}
        # Initialize an empty dictionary to store the current graph
        current_graph = {}
        random_node_iterator = iter(random_node_order)
        i = -1
        only_deletion = False
        cleaning = 0
        while True:
            i +=1
            if (i > 100) and (not current_graph):
                break
            if ((random.random() < deletion_prob_at_each_step) and (i > 0) and current_graph) or (only_deletion and current_graph):
                deletion = True
                # node deletion
                node = nodes_present_at_the_moment.getRandom()
                neighbors_to_delete = [neighbor for neighbor in current_graph[node]]
                start_time = time.time()
                for neighbor in neighbors_to_delete:
                    if not current_graph[neighbor].remove(node):
                        print("We have a problem Houston")
                    if node == neighbor:
                        continue
                    # start timer for pivot
                    # we now update the pivot clustering
                    if (eta[neighbor] != node) or (eta[node] != neighbor):
                        # no need for update in this case
                        continue
                    # when node may have been the pivot of neighbor
                    if eta[neighbor] == node:
                        eta[neighbor] = neighbor
                        for neighbor_of_neighbor in current_graph[neighbor]:
                            if pi[neighbor_of_neighbor] < pi[eta[neighbor]]:
                                eta[neighbor] = neighbor_of_neighbor
                        if eta[neighbor] == neighbor:
                            pivot_clustering[neighbor] = neighbor
                            for neighbor_of_neighbor in current_graph[neighbor]:
                                if neighbor_of_neighbor == neighbor:
                                    continue
                                pivot_clustering[neighbor_of_neighbor] = neighbor if eta[neighbor_of_neighbor] == neighbor else pivot_clustering[neighbor_of_neighbor]
                        else:
                            pivot_clustering[neighbor] = eta[neighbor] if eta[neighbor] == eta[eta[neighbor]] else neighbor
                    # when eta[node] == neighbor. This neighbor may have been the pivot of node
                    else:
                        eta[node] = node
                        for neighbor_of_neighbor in current_graph[node]:
                            if pi[neighbor_of_neighbor] < pi[eta[node]]:
                                eta[node] = neighbor_of_neighbor
                        if eta[node] == node:
                            pivot_clustering[node] = node
                            for neighbor_of_neighbor in current_graph[node]:
                                if neighbor_of_neighbor == node:
                                    continue
                                pivot_clustering[neighbor_of_neighbor] = node if eta[neighbor_of_neighbor] == node else pivot_clustering[neighbor_of_neighbor]
                        else:
                            pivot_clustering[node] = eta[node] if eta[node] == eta[eta[node]] else node
                    dynamic_pivot_total_time += (time.time() - start_time)
                    dynamic_pivot_worst_update_time = max(dynamic_pivot_worst_update_time, time.time() - start_time)

                nodes_present_at_the_moment.remove(node)

                del current_graph[node]
            else:
                deletion = False
                try:
                    # node addition
                    node = next(random_node_iterator)
                except StopIteration:
                    # StopIteration is raised when the iterator is exhausted
                    only_deletion = True
                    continue
                current_graph[node] = OptList()
                nodes_present_at_the_moment.append(node)
                # Add edges to previously arrived nodes
                for neighbor in graph_adjacency_lists[node]:
                    if neighbor in nodes_present_at_the_moment:
                        # I have to update the pivot clustering
                        start_time = time.time()
                        # this is the case where node may become the new pivot of neighbor
                        if pi[eta[neighbor]] > pi[node]:
                            if eta[neighbor] == neighbor:
                                for w in current_graph[neighbor]:
                                    if w == neighbor:
                                        continue
                                    pivot_clustering[w] = w if eta[w]==neighbor else pivot_clustering[w]
                            eta[neighbor] = node
                            pivot_clustering[neighbor] = node if eta[node]==node else neighbor

                        if pi[eta[node]] > pi[neighbor]:
                            if eta[node] == node:
                                for w in current_graph[node]:
                                    if w == node:
                                        continue
                                    pivot_clustering[w] = w if eta[w]==node else pivot_clustering[w]
                            eta[node] = neighbor
                            pivot_clustering[node] = neighbor if eta[neighbor]==neighbor else node
                        dynamic_pivot_total_time += (time.time() - start_time)
                        dynamic_pivot_worst_update_time = max(dynamic_pivot_worst_update_time, time.time() - start_time)
                        current_graph[neighbor].append(node)
                        current_graph[node].append(neighbor)

                # --------------------------------------Notify--------------------------------------
                # ---------send Type 0 notifications-------------------
            start_time = time.time()
            notified_so_far = set()
            if deletion:
                for v in I_nodes[node]:
                    if v == node:
                        print("We may have a problem Houston")
                    B_nodes[v].remove(node)
                got_type_0_notification = B_nodes[node].getRandom(notifications)
                # we need to remove node, from any forward notification set
                if not got_type_0_notification:
                    continue
                got_type_0_notification.discard(node)
                if not got_type_0_notification:
                    continue
            else:
                notified_so_far.add(node)
                got_type_0_notification = current_graph[node].getRandom(notifications)
                if got_type_0_notification:
                    got_type_0_notification.discard(node)
                    I_nodes[node].update(got_type_0_notification)
                    for v in got_type_0_notification:
                        B_nodes[v].append(node)

            # ---------send Type 1 notifications-------------------
            got_type_1_notification = set()
            for v in got_type_0_notification:
                if v not in current_graph:
                    continue
                # Avoid that the notification graph becomes too dense
                for u in I_nodes[v]:
                    B_nodes[u].remove(v)
                # take another sample 1
                br = current_graph[v].getRandom(notifications)
                I_nodes[v].update(current_graph[v].getRandom(notifications))
                I_nodes[v].discard(v)
                I_nodes[v].discard(node)
                for u in I_nodes[v]:
                    B_nodes[u].append(v)
                    if u not in notified_so_far:
                        got_type_1_notification.add(u)
                        notified_so_far.add(u)

            # ---------send Type 2 notifications-------------------
            got_type_2_notification = set()
            for v in got_type_1_notification:
                if v not in current_graph:
                    continue
                for u in I_nodes[v]:
                    B_nodes[u].remove(v)
                # take another sample 2
                br = current_graph[v].getRandom(notifications)
                I_nodes[v].update(current_graph[v].getRandom(notifications))
                I_nodes[v].discard(v)
                I_nodes[v].discard(node)
                for u in I_nodes[v]:
                    B_nodes[u].append(v)
                    if (u not in notified_so_far) or (u not in got_type_2_notification) :
                        got_type_2_notification.add(u)

            # ---------receive Type 2 notifications-------------------
            for v in got_type_2_notification:
                if v not in current_graph:
                    continue
                # just update the sample
                for u in I_nodes[v]:
                    B_nodes[u].remove(v)
                I_nodes[v] = current_graph[v].getRandom(notifications)
                I_nodes[v].discard(v)
                I_nodes[v].discard(node)
                for u in I_nodes[v]:
                    B_nodes[u].append(v)

            # We check if it is a deletion or addition and initialize the respective
            # variables
            if not deletion:
                sparse_graph[node] = OptList()
                Phi_nodes[node] = set()
            else:
                Phi.discard(node)
                neighbors_to_delete = [neighbor for neighbor in sparse_graph[node]]
                for neighbor in neighbors_to_delete:
                    sparse_graph[neighbor].remove(node)
                    sparse_graph[node].remove(neighbor)
                    Phi_nodes[neighbor].discard(node)
                del sparse_graph[node]
                del Phi_nodes[node]

            # The interesting event set corresponds to the notified_so_far
            for u in notified_so_far:
                if deletion and node == u:
                    continue
                if u not in nodes_present_at_the_moment:
                    continue
                    print("Another problem")
                # if u is in Phi we first delete all its edges to nodes not in Phi
                if u in Phi:
                    neighbors_of_u = [v for v in sparse_graph[u]]
                    for v in neighbors_of_u:
                        if (u == v) or (v in Phi):
                            continue
                        sparse_graph[u].remove(v)
                        sparse_graph[v].remove(u)
                        Phi_nodes[v].discard(u)
                Phi.discard(u)
                # we now implement the Anchor procedure
                anchor_prob = anchor_probability_numerator / len(current_graph[u])
                if (random.random() < anchor_prob) and ProbAHeaviness(u, current_graph, epsilon, num_samples_for_agreement_and_heavyness_calculation):
                    Phi.add(u)
                    for v in current_graph[u]:
                        if ProbAgreement(u, v, current_graph, epsilon, num_samples_for_agreement_and_heavyness_calculation):
                            sparse_graph[u].append(v)
                            sparse_graph[v].append(u)
                            if u != v:
                                Phi_nodes[v].add(u)

                # Now we implement the Clean procedure
                connected_nodes_in_anchor_set = [w for w in Phi_nodes[u]]
                for w in connected_nodes_in_anchor_set:
                    if w not in nodes_present_at_the_moment:
                        Phi_nodes[u].discard(w)
                        #del sparse_graph[w]
                        continue
                    if w == u:
                        continue
                    if ProbAgreement(u, w, current_graph, epsilon, num_samples_for_agreement_and_heavyness_calculation) and ProbAHeaviness(w, current_graph, epsilon, num_samples_for_agreement_and_heavyness_calculation):
                        continue
                    else:
                        cleaning+=1
                        sparse_graph[u].remove(w)
                        sparse_graph[w].remove(u)
                        Phi_nodes[u].discard(w)

                # Now we implement the Connect procedure
                J_u = current_graph[u].getRandom(num_samples_for_connect_procedure)
                for w in J_u:
                    phi_nodes_w = [r for r in Phi_nodes[w]]
                    for r in phi_nodes_w:
                        if r not in nodes_present_at_the_moment:
                            Phi_nodes[w].discard(r)
                            continue
                        if r not in current_graph[u]:
                            continue
                        if ProbAgreement(u, r, current_graph, epsilon, num_samples_for_agreement_and_heavyness_calculation) and ProbAHeaviness(r, current_graph, epsilon, num_samples_for_agreement_and_heavyness_calculation):
                            sparse_graph[u].append(r)
                            sparse_graph[r].append(u)
                            Phi_nodes[u].add(r)
            agreement_total_time+=(time.time() - start_time)
            agreement_worst_update_time = max(agreement_worst_update_time, time.time() - start_time)
            if deletion:
                agreement_total_time_node_deletions+=(time.time() - start_time)
                agreement_worst_update_time_node_deletions = max(agreement_worst_update_time_node_deletions, time.time() - start_time)
            else:
                agreement_total_time_node_additions+=(time.time() - start_time)
                agreement_worst_update_time_node_additions = max(agreement_worst_update_time_node_additions, time.time() - start_time)
            if agreement_worst_update_time > 20.5:
                print(f"Deletion = {deletion}")
            if (i+1) % 500 == 0:
                clustering = connected_components(sparse_graph)
                all_singletons = {element: index for index, element in enumerate(sparse_graph.keys())}
                pivot_clustering_this_step = {u: pivot_clustering[u] for u in sparse_graph.keys()}

                # Calculate correlation values and store them
                corr_clustering = correlation_clustering_value(current_graph, clustering)
                corr_all_singletons = correlation_clustering_value(current_graph, all_singletons)
                corr_pivot_clustering = correlation_clustering_value(current_graph, pivot_clustering_this_step)
                if corr_all_singletons != 0:
                    correlation_values_clustering.append(corr_clustering/corr_all_singletons)
                    correlation_values_all_singletons.append(corr_all_singletons/corr_all_singletons)
                    correlation_values_pivot_clustering.append(corr_pivot_clustering/corr_all_singletons)
    print(f" agreement = {np.mean(correlation_values_clustering)} vs pivot = "
      f"{np.mean(correlation_values_pivot_clustering)} vs singleton = {np.mean(correlation_values_all_singletons)}")

            
        

    print(f" agreement = {np.mean(correlation_values_clustering)} vs pivot = "
      f"{np.mean(correlation_values_pivot_clustering)} vs singleton = {np.mean(correlation_values_all_singletons)}")                        
    print(f"agreement time = {agreement_total_time/float(num_experiments)} vs pivot total time = {dynamic_pivot_total_time/float(num_experiments)}")
    print(f"agreement worst update time = {agreement_worst_update_time} vs pivot worst update time = {dynamic_pivot_worst_update_time}")

    print(f"agreement time for additions = {agreement_total_time_node_additions/float(num_experiments)}")
    print(f"agreement worst update time for additions = {agreement_worst_update_time_node_additions}")

    print(f"agreement time for deletions = {agreement_total_time_node_deletions/float(num_experiments)}")
    print(f"agreement worst update time for deletions = {agreement_worst_update_time_node_deletions}")
    
    print("=========================================================================================================")


number of nodes = 13910, number of edges 3524178==> density = 253.35571531272467
 agreement = 0.6879221860488065 vs pivot = 0.5846112675675879 vs singleton = 1.0
 agreement = 0.6879221860488065 vs pivot = 0.5846112675675879 vs singleton = 1.0
agreement time = 12.047162055969238 vs pivot total time = 0.2792983055114746
agreement worst update time = 0.058423757553100586 vs pivot worst update time = 0.00011515617370605469
agreement time for additions = 7.640567779541016
agreement worst update time for additions = 0.05842471122741699
agreement time for deletions = 4.411747455596924
agreement worst update time for deletions = 0.015746116638183594
number of nodes = 13910, number of edges 1597818==> density = 114.86829618979152
 agreement = 0.6276267248219037 vs pivot = 0.5071627653963694 vs singleton = 1.0
 agreement = 0.6276267248219037 vs pivot = 0.5071627653963694 vs singleton = 1.0
agreement time = 14.810634136199951 vs pivot total time = 0.1801755428314209
agreement worst update time = 

We compare the performace of classical pivot to the solution of the dynamic algorithms on the entire graph

In [ ]:
thresholds = [np.mean(distances)/10.0, np.mean(distances)/15.0, np.mean(distances)/20.0, np.mean(distances)/25.0, np.mean(distances)/30.0]


n, m = distances.shape

num_experiments = 1

for threshold in thresholds:
    adjacency_lists = {}
    # self loops are added because diagonal elements have a value of 1
    for a in range(0,n):
        for b in range(0,m):
            if distances[a,b] > threshold:
                continue
            # Add edge to the adjacency lists
            if a not in adjacency_lists:
                adjacency_lists[a] = OptList()
            adjacency_lists[a].append(b)

            if b not in adjacency_lists:
                adjacency_lists[b] = OptList()
            adjacency_lists[b].append(a)
            
    # calculate the objective of the classical pivot on the entire graph
    classical_pivot_clustering_sol = classical_pivot(graph_adjacency_lists)
    classical_pivot_clustering = correlation_clustering_value(current_graph, classical_pivot_clustering_sol)

    
    # now we have created a graph

    # These are the statistics of nodes and edges
    nodes = len(adjacency_lists.keys())
    edges = sum([len(adjacency_lists[node]) for node in adjacency_lists.keys()])
    density = edges/nodes

    print(f"number of nodes = {nodes}, number of edges {edges}==> density = {density}")

    graph_adjacency_lists = adjacency_lists
    
    # this it to measure the time in each execution
    dynamic_pivot_total_time = .0
    agreement_total_time = .0
    dynamic_pivot_worst_update_time = .0
    agreement_worst_update_time = .0

    agreement_total_time_node_additions = .0
    agreement_total_time_node_deletions = .0

    agreement_worst_update_time_node_additions = .0
    agreement_worst_update_time_node_deletions = .0
    
     # Create lists to store objective values for each dataset
    correlation_values_clustering = []
    correlation_values_all_singletons = []
    correlation_values_pivot_clustering = []


    for t in range(0, num_experiments):
        random.seed(t)

        # Generate a random node arrival order
        random_node_order = generate_random_node_order(graph_adjacency_lists)

        # Initialization for the random variables of dynamic pivot
        # we first contract a random permulation pi: node --> order in the random permulation
        pi = {element: index for index, element in enumerate(generate_random_node_order(graph_adjacency_lists))}
        eta = {key: key for key in pi.keys()}
        pivot_clustering = {key: key for key in pi.keys()}

        #Initializaiton for the random variables of agreement
        sparse_graph = {}
        nodes_present_at_the_moment = OptList()
        Phi = set()
        Phi_nodes = {}
        I_nodes = defaultdict(set)
        B_nodes = {node:OptList() for node in graph_adjacency_lists}
        # Initialize an empty dictionary to store the current graph
        current_graph = {}
        random_node_iterator = iter(random_node_order)
        i = -1
        only_deletion = False
        cleaning = 0
        while True:
            i +=1
            if (i > 100) and (not current_graph):
                break
            if ((random.random() < 1) and (i > 0) and current_graph) or (only_deletion and current_graph):
                deletion = True
                # node deletion
                node = nodes_present_at_the_moment.getRandom()
                neighbors_to_delete = [neighbor for neighbor in current_graph[node]]
                start_time = time.time()
                for neighbor in neighbors_to_delete:
                    if not current_graph[neighbor].remove(node):
                        print("We have a problem Houston")
                    if node == neighbor:
                        continue
                    # start timer for pivot
                    # we now update the pivot clustering
                    if (eta[neighbor] != node) or (eta[node] != neighbor):
                        # no need for update in this case
                        continue
                    # when node may have been the pivot of neighbor
                    if eta[neighbor] == node:
                        eta[neighbor] = neighbor
                        for neighbor_of_neighbor in current_graph[neighbor]:
                            if pi[neighbor_of_neighbor] < pi[eta[neighbor]]:
                                eta[neighbor] = neighbor_of_neighbor
                        if eta[neighbor] == neighbor:
                            pivot_clustering[neighbor] = neighbor
                            for neighbor_of_neighbor in current_graph[neighbor]:
                                if neighbor_of_neighbor == neighbor:
                                    continue
                                pivot_clustering[neighbor_of_neighbor] = neighbor if eta[neighbor_of_neighbor] == neighbor else pivot_clustering[neighbor_of_neighbor]
                        else:
                            pivot_clustering[neighbor] = eta[neighbor] if eta[neighbor] == eta[eta[neighbor]] else neighbor
                    # when eta[node] == neighbor. This neighbor may have been the pivot of node
                    else:
                        eta[node] = node
                        for neighbor_of_neighbor in current_graph[node]:
                            if pi[neighbor_of_neighbor] < pi[eta[node]]:
                                eta[node] = neighbor_of_neighbor
                        if eta[node] == node:
                            pivot_clustering[node] = node
                            for neighbor_of_neighbor in current_graph[node]:
                                if neighbor_of_neighbor == node:
                                    continue
                                pivot_clustering[neighbor_of_neighbor] = node if eta[neighbor_of_neighbor] == node else pivot_clustering[neighbor_of_neighbor]
                        else:
                            pivot_clustering[node] = eta[node] if eta[node] == eta[eta[node]] else node
                    dynamic_pivot_total_time += (time.time() - start_time)
                    dynamic_pivot_worst_update_time = max(dynamic_pivot_worst_update_time, time.time() - start_time)

                nodes_present_at_the_moment.remove(node)

                del current_graph[node]
            else:
                deletion = False
                try:
                    # node addition
                    node = next(random_node_iterator)
                except StopIteration:
                    # StopIteration is raised when the iterator is exhausted
                    only_deletion = True
                    continue
                current_graph[node] = OptList()
                nodes_present_at_the_moment.append(node)
                # Add edges to previously arrived nodes
                for neighbor in graph_adjacency_lists[node]:
                    if neighbor in nodes_present_at_the_moment:
                        # I have to update the pivot clustering
                        start_time = time.time()
                        # this is the case where node may become the new pivot of neighbor
                        if pi[eta[neighbor]] > pi[node]:
                            if eta[neighbor] == neighbor:
                                for w in current_graph[neighbor]:
                                    if w == neighbor:
                                        continue
                                    pivot_clustering[w] = w if eta[w]==neighbor else pivot_clustering[w]
                            eta[neighbor] = node
                            pivot_clustering[neighbor] = node if eta[node]==node else neighbor

                        if pi[eta[node]] > pi[neighbor]:
                            if eta[node] == node:
                                for w in current_graph[node]:
                                    if w == node:
                                        continue
                                    pivot_clustering[w] = w if eta[w]==node else pivot_clustering[w]
                            eta[node] = neighbor
                            pivot_clustering[node] = neighbor if eta[neighbor]==neighbor else node
                        dynamic_pivot_total_time += (time.time() - start_time)
                        dynamic_pivot_worst_update_time = max(dynamic_pivot_worst_update_time, time.time() - start_time)
                        current_graph[neighbor].append(node)
                        current_graph[node].append(neighbor)

                # --------------------------------------Notify--------------------------------------
                # ---------send Type 0 notifications-------------------
            start_time = time.time()
            notified_so_far = set()
            if deletion:
                for v in I_nodes[node]:
                    if v == node:
                        print("We may have a problem Houston")
                    B_nodes[v].remove(node)
                got_type_0_notification = B_nodes[node].getRandom(notifications)
                # we need to remove node, from any forward notification set
                if not got_type_0_notification:
                    continue
                got_type_0_notification.discard(node)
                if not got_type_0_notification:
                    continue
            else:
                notified_so_far.add(node)
                got_type_0_notification = current_graph[node].getRandom(notifications)
                if got_type_0_notification:
                    got_type_0_notification.discard(node)
                    I_nodes[node].update(got_type_0_notification)
                    for v in got_type_0_notification:
                        B_nodes[v].append(node)

            # ---------send Type 1 notifications-------------------
            got_type_1_notification = set()
            for v in got_type_0_notification:
                if v not in current_graph:
                    continue
                # Avoid that the notification graph becomes too dense
                for u in I_nodes[v]:
                    B_nodes[u].remove(v)
                # take another sample 1
                br = current_graph[v].getRandom(notifications)
                I_nodes[v].update(current_graph[v].getRandom(notifications))
                I_nodes[v].discard(v)
                I_nodes[v].discard(node)
                for u in I_nodes[v]:
                    B_nodes[u].append(v)
                    if u not in notified_so_far:
                        got_type_1_notification.add(u)
                        notified_so_far.add(u)

            # ---------send Type 2 notifications-------------------
            got_type_2_notification = set()
            for v in got_type_1_notification:
                if v not in current_graph:
                    continue
                for u in I_nodes[v]:
                    B_nodes[u].remove(v)
                # take another sample 2
                br = current_graph[v].getRandom(notifications)
                I_nodes[v].update(current_graph[v].getRandom(notifications))
                I_nodes[v].discard(v)
                I_nodes[v].discard(node)
                for u in I_nodes[v]:
                    B_nodes[u].append(v)
                    if (u not in notified_so_far) or (u not in got_type_2_notification) :
                        got_type_2_notification.add(u)

            # ---------receive Type 2 notifications-------------------
            for v in got_type_2_notification:
                if v not in current_graph:
                    continue
                # just update the sample
                for u in I_nodes[v]:
                    B_nodes[u].remove(v)
                I_nodes[v] = current_graph[v].getRandom(notifications)
                I_nodes[v].discard(v)
                I_nodes[v].discard(node)
                for u in I_nodes[v]:
                    B_nodes[u].append(v)

            # We check if it is a deletion or addition and initialize the respective
            # variables
            if not deletion:
                sparse_graph[node] = OptList()
                Phi_nodes[node] = set()
            else:
                Phi.discard(node)
                neighbors_to_delete = [neighbor for neighbor in sparse_graph[node]]
                for neighbor in neighbors_to_delete:
                    sparse_graph[neighbor].remove(node)
                    sparse_graph[node].remove(neighbor)
                    Phi_nodes[neighbor].discard(node)
                del sparse_graph[node]
                del Phi_nodes[node]

            # The interesting event set corresponds to the notified_so_far
            for u in notified_so_far:
                if deletion and node == u:
                    continue
                if u not in nodes_present_at_the_moment:
                    continue
                    print("Another problem")
                # if u is in Phi we first delete all its edges to nodes not in Phi
                if u in Phi:
                    neighbors_of_u = [v for v in sparse_graph[u]]
                    for v in neighbors_of_u:
                        if (u == v) or (v in Phi):
                            continue
                        sparse_graph[u].remove(v)
                        sparse_graph[v].remove(u)
                        Phi_nodes[v].discard(u)
                Phi.discard(u)
                # we now implement the Anchor procedure
                anchor_prob = anchor_probability_numerator / len(current_graph[u])
                if (random.random() < anchor_prob) and ProbAHeaviness(u, current_graph, epsilon, num_samples_for_agreement_and_heavyness_calculation):
                    Phi.add(u)
                    for v in current_graph[u]:
                        if ProbAgreement(u, v, current_graph, epsilon, num_samples_for_agreement_and_heavyness_calculation):
                            sparse_graph[u].append(v)
                            sparse_graph[v].append(u)
                            if u != v:
                                Phi_nodes[v].add(u)

                # Now we implement the Clean procedure
                connected_nodes_in_anchor_set = [w for w in Phi_nodes[u]]
                for w in connected_nodes_in_anchor_set:
                    if w not in nodes_present_at_the_moment:
                        Phi_nodes[u].discard(w)
                        #del sparse_graph[w]
                        continue
                    if w == u:
                        continue
                    if ProbAgreement(u, w, current_graph, epsilon, num_samples_for_agreement_and_heavyness_calculation) and ProbAHeaviness(w, current_graph, epsilon, num_samples_for_agreement_and_heavyness_calculation):
                        continue
                    else:
                        cleaning+=1
                        sparse_graph[u].remove(w)
                        sparse_graph[w].remove(u)
                        Phi_nodes[u].discard(w)

                # Now we implement the Connect procedure
                J_u = current_graph[u].getRandom(num_samples_for_connect_procedure)
                for w in J_u:
                    phi_nodes_w = [r for r in Phi_nodes[w]]
                    for r in phi_nodes_w:
                        if r not in nodes_present_at_the_moment:
                            Phi_nodes[w].discard(r)
                            continue
                        if r not in current_graph[u]:
                            continue
                        if ProbAgreement(u, r, current_graph, epsilon, num_samples_for_agreement_and_heavyness_calculation) and ProbAHeaviness(r, current_graph, epsilon, num_samples_for_agreement_and_heavyness_calculation):
                            sparse_graph[u].append(r)
                            sparse_graph[r].append(u)
                            Phi_nodes[u].add(r)
            agreement_total_time+=(time.time() - start_time)
            agreement_worst_update_time = max(agreement_worst_update_time, time.time() - start_time)
            if deletion:
                agreement_total_time_node_deletions+=(time.time() - start_time)
                agreement_worst_update_time_node_deletions = max(agreement_worst_update_time_node_deletions, time.time() - start_time)
            else:
                agreement_total_time_node_additions+=(time.time() - start_time)
                agreement_worst_update_time_node_additions = max(agreement_worst_update_time_node_additions, time.time() - start_time)
            if agreement_worst_update_time > 20.5:
                print(f"Deletion = {deletion}")
            if (i+1) % 500 == 0:
                clustering = connected_components(sparse_graph)
                all_singletons = {element: index for index, element in enumerate(sparse_graph.keys())}
                pivot_clustering_this_step = {u: pivot_clustering[u] for u in sparse_graph.keys()}

                # Calculate correlation values and store them
                corr_clustering = correlation_clustering_value(current_graph, clustering)
                corr_all_singletons = correlation_clustering_value(current_graph, all_singletons)
                corr_pivot_clustering = correlation_clustering_value(current_graph, pivot_clustering_this_step)
                if corr_all_singletons != 0:
                    correlation_values_clustering.append(corr_clustering/corr_all_singletons)
                    correlation_values_all_singletons.append(corr_all_singletons/corr_all_singletons)
                    correlation_values_pivot_clustering.append(corr_pivot_clustering/corr_all_singletons)
    print(f" agreement = {np.mean(correlation_values_clustering)} vs pivot = "
      f"{np.mean(correlation_values_pivot_clustering)} vs singleton = {np.mean(correlation_values_all_singletons)}")

            
        

    print(f" agreement = {np.mean(correlation_values_clustering)} vs pivot = "
      f"{np.mean(correlation_values_pivot_clustering)} vs singleton = {np.mean(correlation_values_all_singletons)}")                        
    print(f"agreement time = {agreement_total_time/float(num_experiments)} vs pivot total time = {dynamic_pivot_total_time/float(num_experiments)}")
    print(f"agreement worst update time = {agreement_worst_update_time} vs pivot worst update time = {dynamic_pivot_worst_update_time}")

    print(f"agreement time for additions = {agreement_total_time_node_additions/float(num_experiments)}")
    print(f"agreement worst update time for additions = {agreement_worst_update_time_node_additions}")

    print(f"agreement time for deletions = {agreement_total_time_node_deletions/float(num_experiments)}")
    print(f"agreement worst update time for deletions = {agreement_worst_update_time_node_deletions}")
    
    print("=========================================================================================================")
